## LDA (Linear Discriminant Analysis). Сравнение LDA и PCA. Тематическое моделирование.

In [ ]:
#запустить эту ячейку до начала занятия

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import  CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA,LatentDirichletAllocation
import matplotlib.pyplot as plt

#### Главное отличие LDA от PCA: LDA - это алгоритм обучения с учителем, а PCA - нет.

PCA находит направления с максимальной дисперсией и проектирует данные на эти направления.
LDA создает новую ось таким образом, что при проецировании данных на эту ось объекты двух классов максимально разделяются.

In [ ]:
Iris = load_iris()

data = Iris.data
target = Iris.target
target_names = Iris.target_names

In [ ]:
df = pd.DataFrame(data=np.concatenate((data,target.reshape(150,1)),axis=1),\
                  columns=['col_1','col_2','col_3','col_4','target'])
df.head()

In [ ]:
df.drop(columns=['target'], axis=1, inplace=True)

In [ ]:
pca = PCA(n_components=2)
X_feature_reduced = pca.fit(df).transform(df)

In [ ]:
plt.scatter(X_feature_reduced[:,0], X_feature_reduced[:,1], c=target)
plt.title("PCA")
plt.show()

In [ ]:
lda = LatentDirichletAllocation(n_components=2)

X_feature_reduced2 = lda.fit(df, target).transform(df)

In [ ]:
plt.scatter(X_feature_reduced2[:,0], X_feature_reduced2[:,1], c=target)
plt.title('LDA')
plt.show()

**Наблюдение**

Мы видим, что LDA проектирует данные на такую новую ось, что классы максимально разделены.

### Тематическое моделирование (topic modelling)

Тематическое моделирование - это присваивание темы (topic) каждому документу. Каждая тема представлена определенными словами.

Рассмотрим пример:

У нас есть два топика: топик 1 и топик 2. Топик1 представлен словами "apple, banana, mange",
топик2 - словами "tennis, cricket, hockey". Можем предположить, что в топике1 речь идет о фруктах, а в топике2 - о спорте. Затем каждому новому документу мы присваиваем одну из этих тем (топик1 или топик2).

Другой пример: предположим, у нас есть 6 документов

apple banana
apple orange
banana orange
tiger cat
tiger dog
cat dog

Что будет происходить с тематическим моделированием, если мы захотим извлечь две темы (два топика) из этих документов?
Мы получим два распределения: распределение тема-слово (topic-word) и распределение документ-тема (doc-topic).

Идеальное распределение документ-слово в данном примере будет таким:

![How](df1.png)

Идеальное распределение документ-тема будет таким:

![How](df2.png)

Предположим, что у нас есть новый документ "cat dog apple", тогда его представление по темам должно быть следующим:

Topic1: 0.33

Topic2: 0.63

LDA широко применяется в таких задачах. Его использование для тематического моделирования продемонстрировано ниже. 

Мы подаем на вход LDA число тем (topics), которые хотим выделить в корпусе. 

Но сначала необходимо векторизовать слова (будем использовать подход - мешок слов), поэтому взаимосвязь между словами в текстах при таком подходе исчезнет.

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
lemmatizer = WordNetLemmatizer() #For words Lemmatization
stemmer = PorterStemmer()  #For stemming words
stop_words = set(stopwords.words('english'))

In [ ]:
def TokenizeText(text):
    ''' 
     Tokenizes text by removing various stopwords and lemmatizing them
    '''
    text=re.sub('[^A-Za-z0-9\s]+', '', text)
    word_list=word_tokenize(text)
    word_list_final=[]
    
    for word in word_list:
        if word not in stop_words:
            word_list_final.append(lemmatizer.lemmatize(word))
    return word_list_final

In [ ]:
def gettopicwords(topics, cv, n_words=10):
    '''
        Print top n_words for each topic.
        cv=Countvectorizer
    '''
    for i, topic in enumerate(topics):
        top_words_array = np.array(cv.get_feature_names())[np.argsort(topic)[::-1][:n_words]]
        print("For  topic {} it's top {} words are ".format(str(i),str(n_words)))
             
        combined_sentence=""
        for word in top_words_array:
            combined_sentence+=word+" "
        print(combined_sentence)
#        print(")

In [ ]:
df = pd.read_csv('million-headlines.zip',usecols=[1])
df = df.iloc[:100000]

Data link:

https://www.kaggle.com/therohk/million-headlines

In [ ]:
print(len(df))
df.head()

In [ ]:
%%time 

num_features = 100000
# cv=CountVectorizer(min_df=0.01,max_df=0.97,tokenizer=TokenizeText,max_features=num_features)
cv = CountVectorizer(tokenizer=TokenizeText, max_features=num_features)
transformed_data = cv.fit_transform(df['headline_text'])

In [ ]:
transformed_data

In [ ]:
%%time
no_topics=10  ## We can change this, hyperparameter
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', \
                                learning_offset=50.,random_state=0,n_jobs=-1).fit(transformed_data)

Lda.components_ - это таблица тема-слово, она показывает, какими словами представлена каждая тема.

In [ ]:
gettopicwords(lda.components_,cv)

Присваивание темы документу

Можно заметить, что каждый документ содержит комбинацию тем. Посмотрим на темы первых десяти документов.

In [ ]:
docs = df['headline_text'][:10]

In [ ]:
data = []
for doc in docs:
    data.append(lda.transform(cv.transform([doc])))

In [ ]:
cols = ['topic'+str(i) for i in range(1,11)]
doc_topic_df = pd.DataFrame(columns=cols, data=np.array(data).reshape((10,10)))

In [ ]:
doc_topic_df['major_topic'] = doc_topic_df.idxmax(axis=1)
doc_topic_df['raw_doc'] = docs

In [ ]:
doc_topic_df

Мы увидели, как LDA может быть использован для тематического моделирования. Такой подход также может быть применен для кластеризации документов, основанной на группировке по темам.

Ссылки

https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

https://sebastianraschka.com/faq/docs/lda-vs-pca.html